# Unimaginable Things in Python

**Quirky things that might trip your Python experience if you are not careful**

In [1]:
from pprint import pprint

## Comprehension Syntax
Example of **List Comprehension**:

In [2]:
values = [5, 2, 4, 1, 8, 7, 3, 6]
squared_values = [ x**2 for x in values ]
pprint(squared_values)

[25, 4, 16, 1, 64, 49, 9, 36]


The above comprehension syntax can be desugared into:

In [3]:
squared_values_alt = list( x**2 for x in values )
pprint(squared_values_alt)

[25, 4, 16, 1, 64, 49, 9, 36]


### Decomposing the comprehension syntax

Breaking down this line of code:
```python
squared_values_alt = list( x**2 for x in values )
x = 10
```
A step-by-step break-down:

In [4]:
expr_01 = ( x**2 for x in values )  # an iterable
expr_02 = list(expr_01)  # list takes any iterable
pprint(expr_02)

[25, 4, 16, 1, 64, 49, 9, 36]


### Do we always need a list?
Sometimes, **comprehension-style generator expression** suffices.

In [5]:
squared_values_iterable = ( x**2 for x in values )  # generator expression
for i, v in enumerate(squared_values_iterable, start=1):
    print(f'{i:2d}: {v}')  # f-string since python 3.6

 1: 25
 2: 4
 3: 16
 4: 1
 5: 64
 6: 49
 7: 9
 8: 36


- <span class="hl">😀 **saves memory** — preventing unnecessary memory allocation for intermediate lists.</span>

In [6]:
values = [5, 2, 4, 1, 8, 7, 3, 6]
sq_values = ( x**2 for x in values )
odd_sq_values = [ x for x in sq_values if x % 2 == 1 ]
even_sq_values = [ x for x in sq_values if x % 2 == 0 ]

In [7]:
pprint(odd_sq_values)
pprint(even_sq_values)

[25, 1, 49, 9]
[]


- <span class="hl">Iterable `sq_values` is already **consumed** after its first iteration.</span>